In [1]:
import numpy as np
import pandas as pd
import datetime
import pyreadr
import pandas as pd
from IPython.display import display

from sklearn.preprocessing import OneHotEncoder

from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Unified COVID-19 Dataset

In [2]:
policies_df = pyreadr.read_r('Policy.RData') # also works for Rds
print(policies_df.keys()) # let’s check what objects we got

odict_keys(['Policy'])


In [3]:
policies_df =policies_df['Policy']

In [4]:
policies_df.head()

,ID,Date,PolicyType,PolicyValue,PolicyFlag,PolicyNotes,PolicySource
0,AD,2020-01-01,C1,0.0,NaN,NaN,OxCGRT
1,AD,2020-01-01,C2,0.0,NaN,NaN,OxCGRT
2,AD,2020-01-01,C3,0.0,NaN,NaN,OxCGRT
3,AD,2020-01-01,C4,0.0,NaN,NaN,OxCGRT
4,AD,2020-01-01,C5,0.0,NaN,NaN,OxCGRT


# Coronanet dataset

In [5]:
# import data
df = pd.read_csv('coronanet_release_allvars.csv', encoding='mac_roman')

In [6]:
# colonnes
df.columns

Index(['Unnamed: 0', 'record_id', 'policy_id', 'entry_type', 'correct_type',
       'update_type', 'update_level', 'description', 'date_announced',
       'date_start', 'date_end', 'country', 'ISO_A3', 'ISO_A2',
       'init_country_level', 'domestic_policy', 'province', 'city', 'type',
       'type_sub_cat', 'type_text', 'institution_status', 'target_country',
       'target_geog_level', 'target_region', 'target_province', 'target_city',
       'target_other', 'target_who_what', 'target_direction',
       'travel_mechanism', 'compliance', 'enforcer', 'index_high_est',
       'index_med_est', 'index_low_est', 'index_country_rank', 'link',
       'date_updated', 'recorded_date', 'confirmed_cases', 'deaths',
       'recovered', 'new_tests', 'total_tests', 'total_tests_per_thousand',
       'tests_units', 'Rank_FP', 'Score_FP', 'state_IDC', 'muni_IDC',
       'dispersive_IDC', 'constraining_IDC', 'inclusive_IDC', 'sfi_SFI',
       'ti_cpi_TI', 'pop_WDI_PW', 'gdp_WDI_PW', 'gdppc_WDI_PW',
 

In [7]:
# NaNs
df.isnull().sum()

Unnamed: 0                      0
record_id                       0
policy_id                       0
entry_type                      0
correct_type                    0
update_type                 34546
update_level                36539
description                     0
date_announced                 14
date_start                      0
date_end                    23998
country                         0
ISO_A3                        243
ISO_A2                        267
init_country_level            162
domestic_policy                 0
province                    20635
city                        43546
type                            0
type_sub_cat                13819
type_text                   43425
institution_status          32395
target_country               2159
target_geog_level           35938
target_region               45277
target_province             21264
target_city                 43083
target_other                45640
target_who_what             11137
target_directi

In [8]:
df_2 = df[['policy_id','entry_type','correct_type', 
           'description', 'date_start','date_end','country', 
           'type', 'ISO_A3','init_country_level', 'domestic_policy',
           'type_sub_cat', 'compliance', 'enforcer']]

In [9]:
df_2.shape

(45640, 14)

In [10]:
df_2.isnull().sum()

policy_id                 0
entry_type                0
correct_type              0
description               0
date_start                0
date_end              23998
country                   0
type                      0
ISO_A3                  243
init_country_level      162
domestic_policy           0
type_sub_cat          13819
compliance              244
enforcer                154
dtype: int64

In [11]:
# commençons par le type (mesures plus simples pour les restrictions pour commencer et possibilité d'encodage)
print(df_2['type'].unique())
print(df_2['type'].nunique())

['Anti-Disinformation Measures' 'Closure and Regulation of Schools'
 'Curfew' 'Declaration of Emergency' 'External Border Restrictions'
 'Health Monitoring' 'Health Resources' 'Health Testing' 'Hygiene'
 'Internal Border Restrictions' 'Lockdown'
 'New Task Force, Bureau or Administrative Configuration'
 'Other Policy Not Listed Above' 'Public Awareness Measures' 'Quarantine'
 'Restriction and Regulation of Businesses'
 'Restriction and Regulation of Government Services'
 'Restrictions of Mass Gatherings' 'Social Distancing']
19


## Encodage des données compliance

In [12]:
for _ in df_2['compliance'].unique():
    print('-'*50)
    print(_)

--------------------------------------------------
Voluntary/Recommended but No Penalties
--------------------------------------------------
Mandatory (Unspecified/Implied)
--------------------------------------------------
Mandatory with Exceptions (Please list exceptions in the text entry)
--------------------------------------------------
nan
--------------------------------------------------
Mandatory with Legal Penalties (Jail Time)
--------------------------------------------------
Mandatory with Legal Penalties (Jail Time),Mandatory with Fines
--------------------------------------------------
Mandatory with Fines
--------------------------------------------------
Mandatory (Unspecified/Implied),Mandatory (Other; please list other enforcement mechanisms in the text entry)
--------------------------------------------------
Mandatory with Legal Penalties (Jail Time),Mandatory (Unspecified/Implied)
--------------------------------------------------
Mandatory with Legal Penalties (J

In [13]:
mask = df_2['compliance'].value_counts() > 1000

In [14]:
df_2 = df_2[df_2['compliance'].isin(mask.index)]

In [15]:
df_2.head()

,policy_id,entry_type,correct_type,description,date_start,date_end,country,type,ISO_A3,init_country_level,domestic_policy,type_sub_cat,compliance,enforcer
0,2975738,new_entry,original,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,NaN,Afghanistan,Anti-Disinformation Measures,AFG,National,0,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp..."
1,7024226,new_entry,original,"March 16, In Afghanistan the Government Media ...",2020-03-16,NaN,Afghanistan,Anti-Disinformation Measures,AFG,National,0,NaN,Mandatory (Unspecified/Implied),"National Government,New (COVID-19 specific) Ta..."
2,889484,new_entry,original,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,NaN,Afghanistan,Anti-Disinformation Measures,AFG,National,1,NaN,Voluntary/Recommended but No Penalties,"National Government,Other (Please specify in t..."
3,129322,new_entry,original,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,NaN,Afghanistan,Anti-Disinformation Measures,AFG,National,0,NaN,Voluntary/Recommended but No Penalties,"National Government,Other (Please specify in t..."
4,6721435,new_entry,correction,"""All schools in Herat ñ including temporarily ...",2020-03-09,NaN,Afghanistan,Closure and Regulation of Schools,AFG,Municipal,0,Preschool or childcare facilities (generally f...,Mandatory (Unspecified/Implied),Provincial/State Government


In [16]:
def handle_compliance(x):
    if 'Voluntary/Recommended' in x:
        return 0
    if 'Mandatory (Unspecified/Implied)' in x:
        return 1
    if 'Mandatory with Fines' in x:
        return 2
    if 'Mandatory with Exceptions' in x:
        return 3
    if 'Mandatory with Legal Penalties' in x:
        return 4

In [17]:
df_2['n_compliance'] = df_2['compliance'].map(handle_compliance)

In [18]:
df_2['n_compliance'].nunique()

5

## Encode type colonne

In [19]:
df_2['type'].unique()

array(['Anti-Disinformation Measures',
       'Closure and Regulation of Schools', 'Curfew',
       'Declaration of Emergency', 'External Border Restrictions',
       'Health Monitoring', 'Health Resources', 'Health Testing',
       'Hygiene', 'Internal Border Restrictions', 'Lockdown',
       'New Task Force, Bureau or Administrative Configuration',
       'Other Policy Not Listed Above', 'Public Awareness Measures',
       'Quarantine', 'Restriction and Regulation of Businesses',
       'Restriction and Regulation of Government Services',
       'Restrictions of Mass Gatherings', 'Social Distancing'],
      dtype=object)

In [20]:
df_3 = pd.get_dummies(df_2, columns=["type"]).head()

In [21]:
df_3.columns

Index(['policy_id', 'entry_type', 'correct_type', 'description', 'date_start',
       'date_end', 'country', 'ISO_A3', 'init_country_level',
       'domestic_policy', 'type_sub_cat', 'compliance', 'enforcer',
       'n_compliance', 'type_Anti-Disinformation Measures',
       'type_Closure and Regulation of Schools', 'type_Curfew',
       'type_Declaration of Emergency', 'type_External Border Restrictions',
       'type_Health Monitoring', 'type_Health Resources',
       'type_Health Testing', 'type_Hygiene',
       'type_Internal Border Restrictions', 'type_Lockdown',
       'type_New Task Force, Bureau or Administrative Configuration',
       'type_Other Policy Not Listed Above', 'type_Public Awareness Measures',
       'type_Quarantine', 'type_Restriction and Regulation of Businesses',
       'type_Restriction and Regulation of Government Services',
       'type_Restrictions of Mass Gatherings', 'type_Social Distancing'],
      dtype='object')

## Handle missing value for ISO_3

In [22]:
df_2['ISO_A3'].unique()

array(['AFG', 'ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'AUS',
       'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ',
       'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA',
       'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CAF', 'TCD', 'CHL', 'CHN',
       'COL', 'COM', 'CRI', nan, 'HRV', 'CUB', 'CYP', 'CZE', 'COD', 'DNK',
       'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST',
       'SWZ', 'ETH', 'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU',
       'GHA', 'GRC', 'GRD', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND',
       'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR',
       'ITA', 'CIV', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', '-', 'KWT',
       'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LIE', 'LTU',
       'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL', 'MRT',
       'MUS', 'MEX', 'FSM', 'MDA', 'MCO', 'MNG', 'MNE', 'MAR', 'MOZ',
       'MMR', 'NAM', 'NRU', 'NPL', 'NLD', 'NZL', 'NIC', 'NER', 'NGA',
       'PR

In [23]:
df_2['ISO_A3'] = np.where(df_2['ISO_A3'] == '-', df_2['country'], df_2['ISO_A3'])

In [24]:
df_2['ISO_A3'] = df_2['ISO_A3'].replace('Northern Cyprus', 'CYP')
df_2['ISO_A3'] = df_2['ISO_A3'].replace('Kosovo', 'XK')

In [29]:
df_2['ISO_A3'].unique()

array(['AFG', 'ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'AUS',
       'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ',
       'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA',
       'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CAF', 'TCD', 'CHL', 'CHN',
       'COL', 'COM', 'CRI', nan, 'HRV', 'CUB', 'CYP', 'CZE', 'COD', 'DNK',
       'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST',
       'SWZ', 'ETH', 'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU',
       'GHA', 'GRC', 'GRD', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND',
       'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR',
       'ITA', 'CIV', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'XK',
       'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LIE',
       'LTU', 'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL',
       'MRT', 'MUS', 'MEX', 'FSM', 'MDA', 'MCO', 'MNG', 'MNE', 'MAR',
       'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD', 'NZL', 'NIC', 'NER',
       'NGA', 'P

In [30]:
df_2['ISO_A3'].value_counts()

RUS    5716
USA    5058
DEU    2279
CHE    1880
JPN    1770
ARG    1529
NGA    1111
BRA    1085
CAN     985
IDN     954
IND     891
AUS     674
CHN     669
ESP     417
DZA     416
BGD     413
MEX     358
NZL     352
AFG     347
QAT     346
ITA     330
KAZ     318
MMR     317
AUT     316
ISR     301
LBN     297
ARE     295
FRA     291
GTM     284
AZE     284
       ... 
MLT      35
LTU      34
BFA      33
DJI      32
NER      31
STP      31
PRY      31
ZMB      30
SRB      29
COD      27
KIR      27
NAM      23
COG      23
MWI      21
XK       21
KHM      20
FSM      20
DMA      20
CAF      20
LBR      18
SOM      18
SYR      17
MOZ      16
TUV      15
SEN      15
CPV      14
GIN      12
SLE      12
LCA       8
MHL       7
Name: ISO_A3, Length: 197, dtype: int64

 ## Vectorization des données description (pour plus tard)

In [43]:
def clean(x):
    x = x.lower()
    x = ''.join([letter for letter in x if not letter.isdigit()])
    for p in string.punctuation:
        x = x.replace(p, " ")
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(x)
    tokens = ' '.join([token for token in tokens if token not in stops])
    return tokens

In [44]:
# données description
df_2['clean_description'] = df_2.description.apply(clean)
df_2['clean_description'] = df_2['clean_description'].astype('str')

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df_2.clean_description)

In [46]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation()

model = lda.fit(X)

In [42]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        
print_topics(lda, vectorizer)

Topic 0:
[('th', 2055.820366861567), ('announced', 1324.9235842704034), ('may', 1160.5271555410734), ('pm', 1031.0029476034208), ('march', 993.8825438854873), ('curfew', 984.5746762321426), ('government', 975.3389381760838), ('july', 897.124124920568), ('flights', 861.8646996272723), ('june', 828.4589619059025)]
Topic 1:
[('april', 3566.690983795573), ('may', 2683.239474484084), ('march', 2676.768202157146), ('policy', 1578.0992207414836), ('extended', 1552.3131141089755), ('services', 1453.6217520246023), ('canton', 1346.5291217504641), ('june', 1271.1659698754959), ('allowed', 1030.4733587444134), ('starting', 1016.7184391565813)]
Topic 2:
[('quarantine', 2758.640111263475), ('march', 2719.606131337474), ('people', 2684.400657611952), ('health', 2464.1219746106835), ('days', 1977.6505977055501), ('home', 1870.3234504847924), ('travel', 1606.5759321526673), ('covid', 1438.88879472093), ('countries', 1314.6344710285885), ('citizens', 1252.6030412025523)]
Topic 3:
[('social', 5509.13746

## Check type_sub_cat

In [34]:
for _ in df_2['type_sub_cat'].unique():
    print('-'*50)
    print(_)

--------------------------------------------------
nan
--------------------------------------------------
Preschool or childcare facilities (generally for children ages 5 and below)
--------------------------------------------------
Primary Schools (generally for children ages 10 and below)
--------------------------------------------------
Secondary Schools (generally for children ages 10 to 18)
--------------------------------------------------
Higher education institutions (i.e. degree granting institutions)
--------------------------------------------------
Other External Border Restriction
--------------------------------------------------
Health Certificates
--------------------------------------------------
Temporary Medical Centers
--------------------------------------------------
Other Health Infrastructure
--------------------------------------------------
Temporary Quarantine Centers
--------------------------------------------------
Masks
----------------------------------

[0        False
 1        False
 2        False
 3        False
 4        False
 5        False
 6        False
 7        False
 8        False
 9        False
 10       False
 11       False
 12       False
 13       False
 14       False
 15       False
 16       False
 17       False
 18       False
 19       False
 20       False
 21       False
 22       False
 23       False
 24       False
 25       False
 26       False
 27       False
 28       False
 29       False
          ...  
 45610    False
 45611    False
 45612    False
 45613    False
 45614    False
 45615    False
 45616    False
 45617    False
 45618    False
 45619    False
 45620    False
 45621    False
 45622    False
 45623    False
 45624    False
 45625    False
 45626    False
 45627    False
 45628    False
 45629    False
 45630    False
 45631    False
 45632    False
 45633    False
 45634    False
 45635    False
 45636    False
 45637    False
 45638    False
 45639    False
 Name: type_sub_cat, Len